In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 8 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True)

In [5]:
#import pandas as pd
import numpy as np
def my_generator(x, y, batch_size):
    while True:
        for i in range(0, len(y), batch_size):
            img = x[i:i + batch_size]
            #print(img.shape)
            l = y[i:i + batch_size]
            aug_img, aug_l = next(data_generator.flow(img, l, batch_size=batch_size))
            #print(aug_img.shape)
            images = np.concatenate((img, aug_img), axis=0)
            
            labels = np.concatenate((l, aug_l), axis=0)
            yield images, labels

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

train_generator = my_generator(x_train, y_train, batch_size)

history = model.fit_generator(train_generator,
                    steps_per_epoch=int(len(x_train)/batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Epoch 1/8
390/390 [==============================] - 36s 93ms/step - loss: 1.8536 - acc: 0.3254 - val_loss: 1.3997 - val_acc: 0.4906
Epoch 2/8
390/390 [==============================] - 32s 83ms/step - loss: 1.4511 - acc: 0.4817 - val_loss: 1.0665 - val_acc: 0.6312
Epoch 3/8
390/390 [==============================] - 32s 81ms/step - loss: 1.2646 - acc: 0.5526 - val_loss: 0.9901 - val_acc: 0.6459
Epoch 4/8
390/390 [==============================] - 32s 81ms/step - loss: 1.1440 - acc: 0.5982 - val_loss: 0.9129 - val_acc: 0.6825
Epoch 5/8
390/390 [==============================] - 32s 82ms/step - loss: 1.0537 - acc: 0.6321 - val_loss: 0.7796 - val_acc: 0.7310
Epoch 6/8
390/390 [==============================] - 32s 82ms/step - loss: 0.9852 - acc: 0.6562 - val_loss: 0.7456 - val_acc: 0.7464
Epoch 7/8
390/390 [==============================] - 32s 82ms/step - loss: 0.9341 - acc: 0.6748 - val_loss: 0.7384 - val_acc: 0.7469
Epoch 8/8
390/390 [==